# PySpark RDD API
Ejercicios.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 56.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=1611fe2f0748ed3bf6e95509eed48427a1c19eee0b7d9c57c2ea58f770596ca7
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [2]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#idf = '1PN6TbhTwwl-hVII3P89IMbs1ashJ7BYz'
idf = '1QQ7k1Y5RvL0ZcnQuTVg0rHU6O5AK81zQ'
downloaded = drive.CreateFile({'id':idf})   # replace the id with id of file you want to access
downloaded.GetContentFile('flights.parquet') 
# Fuente del dataset: https://www.kaggle.com/divyansh22/flight-delay-prediction

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
from pyspark.sql import SQLContext, SparkSession

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [0]:
sqlContext = SQLContext(sc)
rdd = sqlContext.read.parquet('flights.parquet').rdd.repartition(8).cache()

In [8]:
rdd.count()

598963

In [9]:
rdd.take(1)

[Row(DAY_OF_MONTH=3, DAY_OF_WEEK=5, OP_UNIQUE_CARRIER='MQ', OP_CARRIER_AIRLINE_ID=20398, OP_CARRIER='MQ', TAIL_NUM='N928AE', OP_CARRIER_FL_NUM=3346, ORIGIN_AIRPORT_ID=13930, ORIGIN_AIRPORT_SEQ_ID=1393007, ORIGIN='ORD', DEST_AIRPORT_ID=11977, DEST_AIRPORT_SEQ_ID=1197705, DEST='GRB', DEP_TIME=1313.0, DEP_DEL15=0.0, DEP_TIME_BLK='1300-1359', ARR_TIME=1400.0, ARR_DEL15=0.0, CANCELLED=0.0, DIVERTED=0.0, DISTANCE=174.0, TIMEDIFF=0)]

In [0]:
# DISTANCE, DEP_TIME, ARR_TIME, DAY_OF_WEEK, ORIGIN, DEST, TAIL_NUM, DAY_OF_MONTH, DAY_OF_WEEK
# Ejemplo rdd.map(lambda x: (x.ORIGIN, x.DEST))

**Ejercicio 1:** Calcular la cantidad de vuelo por línea aérea (usar OP_UNIQUE_CARRIER). Calcular las diez (10) líneas aéreas con mayor cantidad de vuelos. Devolver una lista de Python con los códigos de estas 10 líneas.

In [58]:
vuelos_por_linea_rdd = rdd.map(lambda x: (x[2],1)).reduceByKey(lambda x,y: x+y)
listaDesc = vuelos_por_linea_rdd.takeOrdered(10,lambda x: -x[1])
listaCodigos = []
for x in listaDesc:
  listaCodigos.append(x[0])

listaCodigos

['WN', 'DL', 'AA', 'OO', 'UA', 'YX', 'MQ', 'B6', 'OH', '9E']

**Ejercicio 2:** Calcular el promedio de vuelos que llegaron con 15 minutos de demora o mas (ARR_DEL15 ==1) para las 10 líneas con mas vuelos (usar el ejercicio anterior), de estas indicar las tres mejores y las tres peores.

In [71]:
lineas_demora = rdd.map(lambda x: (x[2],(x[18],1))).filter(lambda x: (listaCodigos.count(x[0]) >= 1) & (x[1][0]==1)).reduceByKey(lambda x,y: (x[0], x[1]+y[1]))
vuelos_lineas_demora = lineas_demora.join(vuelos_por_linea_rdd)
promedio_vuelos_demorados_x_linea = vuelos_lineas_demora.mapValues(lambda x: (x[0][1]/x[1]))
top3_lineas_demoradas = promedio_vuelos_demorados_x_linea.takeOrdered(3,lambda x:-x[1]) 
bottom3_lineas_demoradas = promedio_vuelos_demorados_x_linea.takeOrdered(3,lambda x:x[1])
top3_lineas_demoradas
bottom3_lineas_demoradas

[('MQ', 1.0), ('OO', 1.0), ('WN', 1.0)]

**Ejercicio 3:** Calcular la cantidad de vuelos por ruta. Usando ORIGIN y DEST para la ruta. Devolver un rdd con la siguiente estructura: (RUTA, #Vuelos). Indicar además cuáles son las 10 rutas mas frecuentes y su cantidad de vuelos.

In [22]:
rdd_vuelos_x_ruta = rdd.map(lambda x: ((x[9],x[12]),1)).reduceByKey(lambda x,y: x+y)
rutas_mas_frecuentes = rdd_vuelos_x_ruta.takeOrdered(10, lambda x:-x[1])
rutas_mas_frecuentes

[(('LAX', 'SFO'), 1191),
 (('SFO', 'LAX'), 1189),
 (('LAS', 'LAX'), 1050),
 (('LAX', 'LAS'), 1033),
 (('JFK', 'LAX'), 1023),
 (('ORD', 'LGA'), 1022),
 (('LAX', 'JFK'), 1020),
 (('LGA', 'ORD'), 1015),
 (('HNL', 'OGG'), 941),
 (('OGG', 'HNL'), 939)]

**Ejercicio 4:** Consideremos ahora la cantidad de líneas aéreas que transitan cada ruta, queremos saber cuáles son las diez rutas realizadas por mayor cantidad de líneas aéreas y cuáles son las diez líneas aéreas con mayor cantidad de rutas.
Devolver: Una lista de 10 tuplas de forma (ROUTE, #CARRIERS) y Una lista de 10 tuplas de tipo (Carrier, #ROUTES)

In [27]:
rutas_carriers_rdd = rdd.map(lambda x: ((x[9],x[12],x[2]),1)).reduceByKey(lambda x,y: x+y)
route_carriers = rutas_carriers_rdd.map(lambda x: ((x[0][0],x[0][1]),x[1])).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x:-x[1])
route_carriers

[(('LAX', 'SFO'), 1191),
 (('SFO', 'LAX'), 1189),
 (('LAS', 'LAX'), 1050),
 (('LAX', 'LAS'), 1033),
 (('JFK', 'LAX'), 1023),
 (('ORD', 'LGA'), 1022),
 (('LAX', 'JFK'), 1020),
 (('LGA', 'ORD'), 1015),
 (('HNL', 'OGG'), 941),
 (('OGG', 'HNL'), 939)]

In [28]:
carrier_routes = rutas_carriers_rdd.map(lambda x: (x[0][2],x[1])).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x:-x[1])
carrier_routes

[('WN', 107708),
 ('DL', 79928),
 ('AA', 75472),
 ('OO', 69196),
 ('UA', 48019),
 ('YX', 28826),
 ('MQ', 25284),
 ('B6', 24621),
 ('OH', 23999),
 ('9E', 22955)]

**Ejercicio 5:** Por cada ruta aérea calcular el promedio de tiempo de vuelo. Calculando ARR_TIME - DEP_TIME usando la función provista. Al calculo del tiempo de vuelo en minutos hay que sumarle TIMEDIFF que es la diferencia horaria entre las ciudades (en horas). Por lo tanto el calculo es:

```
hhmmtimediff(x.DEP_TIME,x.ARR_TIME) + (x.TIMEDIFF * 60)
```

Puntos extras: Además del promedio de tiempo de vuelo calcular la desviación standard del tiempo de vuelo para cada ruta.
Devolver: 
- Una lista de 10 tuplas de tipo (ROUTE, average_time)
- Una lista de 10 tuplas de tipo (ROUTE, time_std) (solo para las rutas con mas de 50 vuelos)

In [0]:
from numpy import sqrt
# Computes time diff in format HHMM (in minutes)
def hhmmtimediff(t1, t2):
  m2 = (t2 // 100) * 60 + (t2 % 100)
  m1 = (t1 // 100) * 60 + (t1 % 100)
  return m2 - m1 

In [47]:
tiempo_vuelo_x_ruta_rdd = rdd.map(lambda x: ((x[9],x[12]), (hhmmtimediff(x[13],x[16])+x[21],1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
promedio_tiempo_vuelo_x_ruta_rdd = tiempo_vuelo_x_ruta_rdd.mapValues(lambda x: x[0]/x[1])
promedio_tiempo_vuelo_x_ruta_rdd.take(10)

[(('ORD', 'GRB'), 52.10526315789474),
 (('DAL', 'ABQ'), 45.492753623188406),
 (('OAK', 'ABQ'), 97.3030303030303),
 (('SAN', 'ABQ'), 130.2280701754386),
 (('PIT', 'DCA'), 67.41605839416059),
 (('GSP', 'DCA'), 82.52702702702703),
 (('MSP', 'DCA'), 167.59322033898306),
 (('LAS', 'DCA'), 438.258064516129),
 (('SFO', 'DCA'), 476.2258064516129),
 (('CLT', 'ROC'), 105.05263157894737)]

In [57]:
variacion_vuelo_x_ruta = rdd.map(lambda x: ((x[9],x[12]), (hhmmtimediff(x[13],x[16])+x[21],1))).join(promedio_tiempo_vuelo_x_ruta_rdd).mapValues(lambda x: ((x[0][0]-x[1])**2, x[0][1])).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).mapValues(lambda x: x[0]/(x[1]-1))
variacion_vuelo_x_ruta.take(10)

[(('DAL', 'ABQ'), 65.44885221622765),
 (('GSP', 'DCA'), 91.2115883006294),
 (('CLT', 'ROC'), 126.1142217245241),
 (('ATL', 'ROC'), 22602.48178259562),
 (('JFK', 'IND'), 259.77866698518864),
 (('LGA', 'STL'), 13317.946182439226),
 (('CVG', 'STL'), 51.357894736842105),
 (('SMF', 'STL'), 71.38461538461539),
 (('PHX', 'ELP'), 85207.63292321708),
 (('MCO', 'MEM'), 71.7512077294686)]

**Ejercicio 6:** Para cada linea aerea contar cuantos vuelos tuvieron cuya duracion se excedio en 15 minutos o mas la duracion promedio de la ruta (para todas las lineas). Indicar las 10 mejores lineas aereas y las 10 peores de acuerdo a esta metrica.

In [88]:
#necesito joinear:
#((linea, rutas), duracionVuelo)
#(rutas, tiempo promedio)

#pruebo que el 1ro sea (rutas, (linea,duracionVuelo))
rdd1 = rdd.map(lambda x: ((x[9],x[12]),(x[2],hhmmtimediff(x[13],x[16])+x[21])))
joineados = rdd1.join(promedio_tiempo_vuelo_x_ruta_rdd).map(lambda x: (x[1][0][0], (x[1][0][1]-x[1][1],1))).filter(lambda x: x[1][0]>15).reduceByKey(lambda x,y: (x[0],x[1]+y[1]))
peores = joineados.takeOrdered(10, lambda x: -x[1][1])
mejores = joineados.takeOrdered(10, lambda x: x[1][1])

#print("Mejores lineas aereas")
#for x in mejores:
#  print(x[0])

#print("Peores lineas aereas")
#for x in peores:
#  print(x[0])

[('YV', (64.37619047619047, 6125)),
 ('OO', (20.34210526315789, 20515)),
 ('DL', (17.36585365853658, 39501)),
 ('9E', (70.68478260869566, 5895)),
 ('B6', (69.63709677419355, 12325)),
 ('F9', (45.75630252100841, 6555)),
 ('MQ', (27.684782608695656, 7798)),
 ('AS', (58.34482758620692, 12151)),
 ('G4', (24.55789473684211, 1483)),
 ('HA', (62.853658536585385, 971)),
 ('WN', (54.507246376811594, 37078)),
 ('AA', (67.37619047619047, 39574)),
 ('YX', (15.684782608695656, 11030)),
 ('OH', (17.47297297297297, 9166)),
 ('UA', (17.654008438818565, 25253)),
 ('EV', (49.34210526315789, 3784)),
 ('NK', (44.75630252100841, 9594))]